<a href="https://colab.research.google.com/github/mohamedahmedsaadahmed77/OCR-Selected-2/blob/master/CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!git clone https://github.com/mohamedahmedsaadahmed77/OCR-Selected-2.git

In [0]:
import numpy as np
import pandas as pd
import os
from imutils import paths
import cv2

def load_images(path):
  print("[INFO] loading images...")
  imagePaths = list(paths.list_images(path))
  imagePaths.sort()
  data = []
  labels = []
  for imagePath in imagePaths:
    label = imagePath.split(os.path.sep)[-2]
    image = cv2.imread(imagePath, 0)
    image = np.reshape(image, (28, 28, 1))/255.0
    data.append(image)
    labels.append(label)
  data = np.array(data)
  labels=pd.get_dummies(labels)
  labels = np.array(labels)
  print("[INFO] done")
  return data,labels

In [0]:
import math
import matplotlib.pyplot as plt

def showMeImages(labels, images):
  length = len(images)
  dim = math.ceil(math.sqrt(length))
  for i in range(length):
    plt.subplot(dim,dim,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(images[i], cmap=plt.cm.binary)
    plt.xlabel(labels[i])

In [0]:
from sklearn.model_selection import train_test_split

x, y = load_images('/content/OCR-Selected-2/Dataset')
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.1, random_state=42)

In [0]:
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,Activation,Dropout,Dense,Flatten

model = Sequential()
model.add(Conv2D(64, (3,3), activation='relu', input_shape=(28,28,1)))
model.add(Conv2D(32, kernel_size=3, activation='relu'))
model.add(Flatten())
model.add(Dense(10, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=8, batch_size=256)